# Data Wrangling: Part 1

The following series of notebooks are seperated for convinence and are all associated with Data Wrangling. There are two datasets we are working with for this project. The first dataset is the United Healthcare insurance claims dataset, which was just released. Unfortunately for us, these files are large json files that are not in a format that we can use at this time. In this notebook I hope to obtain the discount rates that we need.

## Import packages

Here I import the packages that I need. All the packages can be downloaded through conda or pip install.

In [1]:
import requests
import shutil
import ijson.backends.python as ijson
import json
import gzip
from tqdm.auto import tqdm
from csv import writer
import os
from io import StringIO
import pandas as pd
import numpy as np
from dotenv import load_dotenv

### Data Organization

To get started, its important to note how the data is available to us. As stated prior, the data is availiable online at: 

https://transparency-in-coverage.uhc.com/?_gl=1*5it7ok*_ga*NjMzOTkzMDA0LjE2NzI3OTc4MjA.*_ga_HZQWR2GYM4*MTY3Mjc5NzgyMC4xLjAuMTY3Mjc5NzgyMC4wLjAuMA

You can get some information about the data is organized through this git repository:

https://github.com/CMSgov/price-transparency-guide


For summation, some basic facts:
1. UHC has organized the files for download by either in-network and out-of-network data sets. Since we are interested is discount rates for those with CMS metrics, we are only going to be working with datasets that are within network, since UHC would not negotiate for out of network rates and these rates are most likely standardized by the provider. I created an excel spreadsheet called json_files_hyperlinks.xlsx with these contained json file hyperlinks.
2. The json files are seperated by insured group, and some groups, that exceptionally large, and split up. For this reason, there are going to be repeated discount rates for various procedures/medications etc.
3. Each json file is divided into two:
    
    a. Provider groups (aka a hospital, pharmacy, or private practice) which have reference numbers to be called
    
        i. NPI: providers unique number
    
        ii. reference number for the provider group
    
    b. Procedure/drug/service (code type and code number)
    
        i. reference number (referring to the group above)
    
        ii. discount rate negotiated with the group


I will break this down futhure as we move along this notebook. For now I am currently using a dotenv to set up my directory where I am accessing files and storying files.


In [2]:
# Define paths to important files and places where you want to store files
load_dotenv()

# This is the file with the hyperlinks of json files we need to parse
hyperlink_path = 'json_files_hyperlinks.xlsx'
hyperlinks = pd.read_excel(hyperlink_path)['Hyperlinks'].tolist()

# Here is a place where I intent to store the json files to be downloaded alongside creating folders for the parsed data: These are large files so this will outside github.
parent_dir= os.getenv('dir')
dir_json = os.path.join(parent_dir, 'JSON')
dir_data = os.path.join(parent_dir,'data_update')

The follow sections of code are used to parse the data. I have broken it up intro steps.

First is downloading the files. Here I simiply take the hyperlink list and try to download the files, name the file without the html address and obtain the size of the file. This is to prevent downloading too large of a file. Included are other functions that might be helpful if you want to download multiple files and if you want to delete the file after it is parsed:

In [ ]:
def download_file(url, path):
    local_filename = url.split('/')[-1]
    download_path = os.path.join(path,local_filename)
    if os.path.exists(download_path):
        print(download_path + '\nFile already Exists!')
        return (local_filename, download_path)
    else:
        with requests.get(url, stream=True) as r:
            total_length = int(r.headers.get('content-length'))
            with tqdm.wrapattr(r.raw, "read", total=total_length, desc="")as raw:
                with open(download_path, 'wb') as output:
                    shutil.copyfileobj(raw, output)
        print(local_filename +'\nDownload Complete')
        return (local_filename, download_path)


def check_file_size(url):
    with requests.get(url, stream=True) as r:
        return int(r.headers.get('content-length'))

def download_multiple_files(urls, path):
    local_filenames = []
    download_paths = []
    for url in urls:
        local_filename = url.split('/')[-1]
        download_path = os.path.join(path,local_filename)
        if os.path.exists(download_path):
            print(download_path + '\nFile already Exists!')
            local_filenames.append(local_filename) 
            download_paths.append(download_path)
        else:
            with requests.get(url, stream=True) as r:
                total_length = int(r.headers.get('content-length'))
                with tqdm.wrapattr(r.raw, "read", total=total_length, desc="")as raw:
                    with open(download_path, 'wb') as output:
                        shutil.copyfileobj(raw, output)
            print(local_filename +'\nDownload Complete!')
            local_filenames.append(local_filename) 
            download_paths.append(download_path)
    return (local_filenames, download_paths)

def delete_file(path):
    os.remove(path)
    print(path + '\nFile Deleted')

This function simply makes a folder name that matchs the hyperlink file to store or CVs after the data is parsed.

In [ ]:
def make_paths_folders(filename, json_file):
    folder_name = filename[0:-8]
    path= os.path.join(dir_data, folder_name)
    if os.path.exists(path) is False:
        os.mkdir(path)
    else:
        print(path + '\nFolder Already Exists')
    return path

The following code is to keep track of what files we have parsed. Since this is a large data set, it might be worth your time to split up the work as the data is parsed. Some of the files might be to large, and so I skipped these data sets.

In [ ]:
def write_large_file(filename, index, hyperlink):
    with open(filename, 'a') as f:
        writer_object = writer(f)
        writer_object.writerow([index,hyperlink])
        print('File Too Large, written to Large File CSV')
        f.close()

def write_completed_file(filename, index, hyperlink):
    with open(filename, 'a') as f:
        writer_object = writer(f)
        writer_object.writerow([index,hyperlink])
        print('File has been completed')
        f.close()

The following code are fuctions related to parsing. The first two functions take in data from the json file and respectively write provider group data and billing code information with rates and right them into different files to store as CSV.

In [3]:
def write_provider_csv(filename, reference, tin, npi_provider_groups):
    with open(filename, 'a') as f:
        writer_object = writer(f)
        for i,r in enumerate(reference):
            writer_object.writerow([r,tin[i],npi_provider_groups[i]])
        f.close()


def write_rates_csv(filename, billing_type, billing_code, provider_reference, rate):
    with open(filename, 'a') as f:
        writer_object = writer(f)
        for i, ref in enumerate(provider_reference):
            writer_object.writerow([billing_type[i],billing_code[i],ref,rate[i]])
        f.close()




The following code is responsible for parsing the json file. It goes through the code line by line and matching the values we are interested in and stores them. The parse_file contains two sections, reference groups and rates.

The last two function combine the processes of the functions we have already wrote.

In [ ]:
def parse_file(filename, json_file):
    path = make_paths_folders(filename, json_file)
    providers_csv = os.path.join(path, filename[0:-8]) + '_providers.csv'
    rates_csv = os.path.join(path, filename[0:-8]) + '_rates.csv'
    write_provider_csv(providers_csv, ['provider_reference'], ['tin'], ['npi_provider_groups'])
    write_rates_csv(rates_csv, ['billing_type'], ['billing_code'], ['provider_reference'], ['negotiated_rates'])

    npi_provider_groups = []
    tin = []
    reference = []

    billing_type = []
    billing_code = []
    ref_group = []
    rates = []

    with gzip.open(json_file, mode="rt") as f:
        parser = ijson.parse(f)
        for prefix, event, value in tqdm(parser):
            if len(npi_provider_groups) >= 1000:
                write_provider_csv(providers_csv, reference, tin, npi_provider_groups)
                npi_provider_groups = []
                tin = []
                reference = []
            if len(rates) >= 10000:
                write_rates_csv(rates_csv, billing_type, billing_code, ref_group, rates)
                billing_type = []
                billing_code = []
                ref_group = []
                rates = []

            
            if prefix=='provider_references.item.provider_groups.item.npi' and event=='start_array'and value==None:
                temp_npi = []
            elif prefix=='provider_references.item.provider_groups.item.npi.item' and event=='number':
                temp_npi.append(value)
            elif prefix=='provider_references.item.provider_groups.item.tin.value' and event=='string':
                temp_tin =value
            elif prefix=='provider_references.item.provider_group_id' and event=='number':
                npi_provider_groups.append(temp_npi)
                tin.append(temp_tin)
                reference.append(value)

            elif prefix=='provider_references' and event=='end_array':
                write_provider_csv(providers_csv, reference, tin, npi_provider_groups)
                npi_provider_groups = []
                tin = []
                reference = []
            
            elif prefix=='in_network.item.billing_code_type' and event=='string':
                temp_type = value
            elif prefix=='in_network.item.billing_code' and event=='string':
                temp_code = value
            elif prefix=='in_network.item.negotiated_rates.item.provider_references.item' and event=='number':
                temp_ref = value
            elif prefix=='in_network.item.negotiated_rates.item.negotiated_prices.item.negotiated_rate' and event=='number':
                billing_type.append(temp_type)
                billing_code.append(temp_code)
                ref_group.append(temp_ref)
                rates.append(value)
            elif prefix=='in_network' and event=='end_array':
                write_rates_csv(rates_csv, billing_type, billing_code, ref_group, rates)
                billing_type = []
                billing_code = []
                ref_group = []
                rates = []
        f.close()
    print(json_file + '\nParse Complete')    

def download_parse(url, path):
    (filename, json_file) = download_file(url, path)
    parse_file(filename, json_file)


def download_parse_delete(url, path):
    (filename, json_file) = download_file(url, path)
    parse_file(filename, json_file)
    delete_file(json_file)

Here you can store the the file you want to access. 

In [16]:
hyperlinks[2949]

'https://uhc-tic-mrf.azureedge.net/public-mrf/2023-01-01/2023-01-01_UnitedHealthcare-Insurance-Company_Insurer_D0015336_UHC-Dental_in-network-rates.json.gz'

Lets work with files that are less than 100MB and parse those only to keep our RAM in check. The following processes may take some time.

In [17]:
number_to_parse = 1000

start = 2949
large_files = 'json_large_hyperlinks_update.csv'
completed = 'json_completed_hyperlinks_update.csv'
for i in range(start,start + number_to_parse):
    if check_file_size(hyperlinks[i]) < 100_000_000: 
        print('Hyperlink File: ' + str(i) + ' Started!')
        download_parse_delete(hyperlinks[i], dir_json)
        write_completed_file(completed, i, hyperlinks[i])
    else:
        write_large_file(large_files,i,hyperlinks[i])
        print('File number: ' + str(i))


Hyperlink File: 2949 Started!


  0%|          | 0/79508575 [00:00<?, ?it/s]

2023-01-01_UnitedHealthcare-Insurance-Company_Insurer_D0015336_UHC-Dental_in-network-rates.json.gz
Download Complete


0it [00:00, ?it/s]

D:/Vignesh/Capstone\JSON\2023-01-01_UnitedHealthcare-Insurance-Company_Insurer_D0015336_UHC-Dental_in-network-rates.json.gz
Parse Complete
D:/Vignesh/Capstone\JSON\2023-01-01_UnitedHealthcare-Insurance-Company_Insurer_D0015336_UHC-Dental_in-network-rates.json.gz
File Deleted
File has been completed
Hyperlink File: 2950 Started!


...

D:/Vignesh/Capstone\JSON\2023-01-01_UnitedHealthcare-Insurance-Company_Insurer_D0030465_UHC-Dental_in-network-rates.json.gz
Parse Complete
D:/Vignesh/Capstone\JSON\2023-01-01_UnitedHealthcare-Insurance-Company_Insurer_D0030465_UHC-Dental_in-network-rates.json.gz
File Deleted
File has been completed
Hyperlink File: 3107 Started!


  0%|          | 0/79508575 [00:00<?, ?it/s]

2023-01-01_UnitedHealthcare-Insurance-Company_Insurer_D0030466_UHC-Dental_in-network-rates.json.gz
Download Complete


0it [00:00, ?it/s]

KeyboardInterrupt: 